In [ ]:
from atmospheric_explorer.cams_interfaces import InversionOptimisedGreenhouseGas
from atmospheric_explorer.shapefile import ShapefilesDownloader
from atmospheric_explorer.utils import get_local_folder
from atmospheric_explorer.units_conversion import convert_units_array
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
from glob import glob
import xarray as xr
from datetime import datetime
import geopandas as gpd
from shapely.geometry import mapping
import rioxarray
import plotly.graph_objects as go
import shutil
import numpy as np
import statsmodels.stats.api as sms
import pandas as pd

In [ ]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'data'))

In [ ]:
# Remove previous data
shutil.rmtree(os.path.join(get_local_folder(), 'shapefiles'))

In [ ]:
#function to move from 0+360 to -180+180 long
def ds_swaplon(ds):
    return ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180)).sortby('longitude')

In [ ]:
manager = InversionOptimisedGreenhouseGas(
    data_variables='carbon_dioxide',
    file_format='zip',
    quantity='surface_flux',
    input_observations='surface',
    time_aggregation='monthly_mean',
    year=[
        '1985', '1986', '1987',
        '1988', '1989', '1990',
        '1991', '1992', '1993',
        '1994', '1995', '1996',
        '1997', '1998', '1999',
        '2000', '2001', '2002',
        '2003', '2004', '2005',
        '2006', '2007', '2008',
        '2009', '2010', '2011',
        '2012', '2013', '2014',
        '2015', '2016', '2017',
        '2018', '2019', '2020'
    ],
    month=[
        '01', '02', '03',
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12'
    ]
)
manager.download()

In [ ]:
files = sorted(list(glob(manager.file_full_path)))

In [ ]:
# Create dataframe with first file
mm = datetime.strptime(files[0].split('_')[-1].split('.')[0], '%Y%m')
df = xr.open_dataset(files[0])[['flux_foss']]
df = df.expand_dims({'time': [mm]})
# Merge remaining files
for file in files[1:]:
    mm = datetime.strptime(file.split('_')[-1].split('.')[0], '%Y%m')
    temp = xr.open_dataset(file)[['flux_foss']]
    temp = temp.expand_dims({'time': [mm]})
    df = xr.combine_by_coords([df, temp])

In [ ]:
df = df.rio.write_crs('EPSG:4326')

In [ ]:
sh_down = ShapefilesDownloader(
    resolution='10m',
    instance='countries_ita'
)
sh_down.download_shapefile()

In [ ]:
sh = gpd.read_file(sh_down.shapefile_full_path, crs='EPSG:4326')

In [ ]:
df_clipped = df.rio.clip(sh[sh['ADMIN'] == 'Italy'].geometry.apply(mapping), sh.crs, drop=True)[['flux_foss']]

In [ ]:
df_clipped['flux_foss'][0].plot()

In [ ]:
# Drop all values that are null over all coords, compute the mean of the remining values over long and lat
df_clipped = df_clipped.where(~df_clipped['flux_foss'].isnull(), drop=True).sortby('time').mean(dim=['longitude', 'latitude'])

In [ ]:
# Cool but not interactive
sns.lineplot(
    y=df_clipped['flux_foss'].values,
    x=df_clipped.coords['time.year']
)

In [ ]:
# Xarray doesn't cover all pandas functionalities, we need to convert it to a pandas dataframe
df_pandas = df_clipped.to_pandas().reset_index()
df_pandas['year'] = df_pandas['time'].dt.year
df_pandas = df_pandas.groupby('year').agg(mean=('flux_foss', 'mean'), ci=('flux_foss', lambda d: sms.DescrStatsW(d).tconfint_mean()))
df_pandas[['lower', 'upper']] = pd.DataFrame(df_pandas['ci'].to_list(), index=df_pandas.index)

In [ ]:
# Plotly plot, it's interactive, some tweaking needed for the theme
times = df_pandas.index.tolist()
times_rev = times[::-1]

# Line 1
y1 = df_pandas['mean'].to_list()
y1_upper = df_pandas['upper'].to_list()
y1_lower = df_pandas['lower'].to_list()
y1_lower = y1_lower[::-1]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=times+times_rev,
    y=y1_upper+y1_lower,
    fill='toself',
    fillcolor='rgba(0,100,200,0.2)',
    line_color='rgba(0,100,200,0.2)'
))
fig.add_trace(go.Scatter(
    x=times,
    y=df_pandas['mean'].to_list(),
    line_color='rgb(0,100,200)'
))
fig.update_traces(mode='lines')
fig.show()

In [ ]:
# TODO:
## 1 - Shiftare lat e long se necessario -> EAC4 va da 0 a 360, ma Inversion è già tra -180+180
## 2 - Clip paese -> Capire se funziona, sembra funzionare
## 3 - Media annuale -> In realtà ci dovrebbe essere un modo per calcolare il CI su plotly, basta avere diversi valori per anno
##                    -> No, quella è seaborn, su plotly va fatto a mano usando ad esempio statsmodels
## 4 - Plot con CI al 95% e aggiugere la seconda linea